# Lab 2 : Decision Tree

## Preparing the dataset

Import necessary libraries and routines.

In [1]:
import os
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report, confusion_matrix
import graphviz

Download the data set.

In [2]:
%pip install ucimlrepo

Note: you may need to restart the kernel to use updated packages.


Load dataset from drive to memory in `pandas.DataFrame`.

In [3]:
from ucimlrepo import fetch_ucirepo
breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17)

feature = breast_cancer_wisconsin_diagnostic.data.features
label = breast_cancer_wisconsin_diagnostic.data.targets

Create 4 copies of the original data set. For each one, split it into two parts: train and test. The propotion of splitting for each copy is: 60/40, 40/60, 80/20, 90/10.

In [4]:
X_trains, y_trains = dict(), dict()
X_tests, y_tests = dict(), dict()

train_test_propotions = ['60/40', '40/60', '80/20', '90/10']

for propotion in train_test_propotions:
    test_size = 1 - int(propotion.split('/')[0]) / 100
    X_train, X_test, y_train, y_test = train_test_split(
        feature, label, 
        test_size=test_size,
        stratify=label)
    X_trains[propotion] = X_train
    y_trains[propotion] = y_train
    X_tests[propotion] = X_test
    y_tests[propotion] = y_test

## Building the tree classifier

Train 4 models and save them along with their accuracies.

In [5]:
models = dict()

for propotion in train_test_propotions:
    model = DecisionTreeClassifier()
    model.fit(X_trains[propotion], y_trains[propotion])
    models[propotion] = model

`export_model_image` function exports visualizing image of the decision tree, with a custom name.

In [6]:
def export_model_image(model, filename, save_picture=True):
    dot_data = export_graphviz(model, 
                            out_file=None, 
                            feature_names=[feature_name for feature_name in feature], 
                            class_names=['benign', 'malignant'],
                            filled=True, 
                            rounded=True,
                            special_characters=True)

    graph = graphviz.Source(dot_data)  
    graph.format = 'png'
    graph.render(filename=filename)
    # graph.view()

Export visualizations of all models. Images are saved in folder with name as `visualization_folder`.

In [7]:
visualization_folder = 'images'

if os.path.exists(visualization_folder) == False:
    os.mkdir(visualization_folder)

for propotion, model in models.items():
    propotion = propotion.replace('/', '_')
    export_model_image(model, f'{visualization_folder}/Decision_Tree_({propotion})')

## Evaluating the decision tree classifiers

Create classification report and confusion matrix for each models.

In [8]:
model_classification_reports = dict()
model_confusion_matrices = dict()

for model_name, model in models.items():
    cls_report = classification_report(y_tests[model_name], model.predict(X_tests[model_name])) 
    conf_matrix = confusion_matrix(y_tests[model_name], model.predict(X_tests[model_name]))
    print(f'Model: {model_name}')
    print(f'Classification Report:\n{cls_report}')
    print(f'Confusion Matrix:\n{conf_matrix}\n')
    model_classification_reports[model_name] = cls_report
    model_confusion_matrices[model_name] = conf_matrix

Model: 60/40
Classification Report:
              precision    recall  f1-score   support

           B       0.97      0.91      0.94       143
           M       0.86      0.95      0.91        85

    accuracy                           0.93       228
   macro avg       0.92      0.93      0.92       228
weighted avg       0.93      0.93      0.93       228

Confusion Matrix:
[[130  13]
 [  4  81]]

Model: 40/60
Classification Report:
              precision    recall  f1-score   support

           B       0.98      0.91      0.94       215
           M       0.87      0.97      0.91       127

    accuracy                           0.93       342
   macro avg       0.92      0.94      0.93       342
weighted avg       0.94      0.93      0.93       342

Confusion Matrix:
[[196  19]
 [  4 123]]

Model: 80/20
Classification Report:
              precision    recall  f1-score   support

           B       0.93      0.96      0.95        72
           M       0.93      0.88      0.90  

### Explanation:

#### `confusion_matrix`

In the confusion matrix $C$, the row represents the true Benign or Malignant class, while the column represents the prediction of the model.
- $C_{0,0}$: Both the truth and the prediction are Benign
- $C_{0,1}$: The truth is Benign, but the prediction is Malignant
- $C_{1,0}$: The truth is Malignant, but the prediction is Benign
- $C_{1,1}$: Both the truth and the prediction are Malignant

#### `classification_report`

The Precision, Recall and F1 Score can be calculated based on which type of class is chosen to be Positive in the case of binary classification (for example: if Malignant as positive, Benign as positive). For more detail, please refer to the report document.

1. precision: **What percentage of all the Positive predictions made by the model were accurate?**

The formula: $$Precision=\frac{True \ Positives}{True \ Positives + False \ Positives}$$

2. recall: **What percentage of all the actual Positives were accurately predicted by the model?**

The formula: $$Recall=\frac{True \ Positive}{True \ Positive + False \ Negative}$$

3. F1 Score: **The harmonic mean of Precision and Recall.** If any of them becomes extremely low, F1 Score will also go down. Thus, F1 Score can help you find a good balance between Precision and Recall.

The formula: $$F1 \ Score=\frac{2 \times Precision \times Recall}{Precision + Recall}$$